In [1]:
%load_ext autoreload

In [3]:
%autoreload 2

In [5]:
# Add the parent directory of 'src' to the Python path
import os
import sys


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [6]:
from src.phrase import *

file_path = '../data/longman-communication-3000.json'
data = load_longman_data(file_path)
s1_words = filter_s1_words(data)


Searching for config.json...
Checking: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\notebooks\config.json
Checking: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\src\config.json
Found config file at: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\src\config.json
Successfully loaded config from: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\src\config.json
Multiple country codes available for en: en-AU, en-GB, en-IN, en-US
Config loader initialized.
Config file location: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\src\config.json
Current working directory: c:\Users\i5\Documents\Python Scripts\audio-language-trainer\notebooks


In [10]:
word_list = [word for words in s1_words.values() for word in words]

In [15]:
from random import shuffle


shuffle(word_list)

In [17]:

phrases = generate_phrases_with_llm(word_list[0:500], num_phrases=100)


In [26]:
phrases.split("\n")

['{',
 '    "phrases": [',
 '        "The lovely couple will watch the sunset together",',
 '        "How did you manage to organize the event",',
 '        "She had to suffer through a difficult exam",',
 '        "They are considering a move to the west",',
 '        "Can you please pass me that blue bottle",',
 '        "We should add more information to the report",',
 '        "The doctor will provide an answer next week",',
 '        "They have been waiting for hours in line",',
 '        "He realized the importance of a good education",',
 '        "The tired student needed a coffee break badly",',
 '        "We ought to pay more attention to health",',
 '        "The wife wondered about their future together",',
 '        "They will build a new office in the centre",',
 '        "She had to break the news to her mother",',
 '        "The minister spoke about the environment yesterday",',
 '        "You must apply for the job by Friday",',
 '        "They are planning to open a 

In [ ]:

print(f"Generated {len(phrases)} phrases:")
for phrase in phrases:
    print(phrase)

print(f"\nUsed {len(used_words)} unique words.")

# Update word usage
data = update_word_usage(data, used_words)


In [ ]:

# Optionally, save the updated data back to the file
with open(file_path, 'w') as file:
    json.dump(data, file, indent=2)